# Example: Multi Agent

Install SUMO.

In [ ]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo

Install SMARTS.

In [ ]:
# Install SMARTS
%cd ~
!rm -rf /content/SMARTS
!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS
!cd /content/SMARTS && git checkout 'ipynb-2' && pip install .[camera-obs]
!echo -e "import sys\nsys.path.insert(0, '/content/SMARTS/')" | python

Build the scenario.

In [ ]:
# Build scenarios
!scl scenario build-all --clean /content/SMARTS/scenarios/loop

Define the agent.

In [ ]:
from smarts.core.agent import Agent

class KeepLaneAgent(Agent):
    def act(self, obs):
        return "keep_lane"

Define the environment loop.

In [ ]:
import gym

from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.core.utils.episodes import episodes

N_AGENTS = 4
AGENT_IDS = ["Agent %i" % i for i in range(N_AGENTS)]

def main(scenarios, num_episodes, max_episode_steps=None):
    agent_specs = {
        agent_id: AgentSpec(
            interface=AgentInterface.from_type(
                AgentType.Laner, max_episode_steps=max_episode_steps
            ),
            agent_builder=KeepLaneAgent,
        )
        for agent_id in AGENT_IDS
    }

    env = gym.make(
        "smarts.env:hiway-v0",
        scenarios=scenarios,
        agent_specs=agent_specs,
        headless=True,
        sumo_headless=True,
    )

    for episode in episodes(n=num_episodes):
        agents = {
            agent_id: agent_spec.build_agent()
            for agent_id, agent_spec in agent_specs.items()
        }
        observations = env.reset()
        episode.record_scenario(env.scenario_log)

        dones = {"__all__": False}
        while not dones["__all__"]:
            actions = {
                agent_id: agents[agent_id].act(agent_obs)
                for agent_id, agent_obs in observations.items()
            }

            observations, rewards, dones, infos = env.step(actions)
            episode.record_step(observations, rewards, dones, infos)

    env.close()

Run the example.

In [ ]:
main(
    scenarios=["/content/SMARTS/scenarios/loop"],
    num_episodes=3,
    max_episode_steps=100,
)